In [44]:
from Bio.Align.Applications import ClustalwCommandline

#coinfig
base_file = '12_otus_97 copy'
in_file = base_file + ".txt"
clustalw_cline = ClustalwCommandline("clustalw2", infile=in_file)


In [45]:
# clustal algorhythm
_,__ = clustalw_cline()

In [47]:
# pick up the order from newick file
def OTU_extractor(string):
    ind = string.find(':')
    return string[:ind]
with open(base_file + '.dnd','r') as f:
    l_strip = [OTU_extractor(s.strip()) for s in f.readlines() if s.strip()[0] == 'O']

In [48]:
# save as pandas frame
import pandas as pd
l_strip_df = pd.DataFrame(l_strip,columns=['order'])
l_strip_df.to_csv(base_file + '_order.csv')

In [56]:
## load data for heatmap
# load order
order_df = pd.read_csv(base_file + '_order.csv').loc[:,'order'].to_list()
# load freq 
freq_df = pd.read_csv('17_otu_97_table_taxonomy_Chl_mito copy.txt', sep="\t",header=1)
freq_df = freq_df.set_index('#OTU ID')
# reorder 
freq_df_ordered = freq_df.reindex(index=order_df)

In [68]:
# calculate density
sum_freq_each_samples = []
for col in freq_df_ordered.columns.to_list():
    if col == 'ConsensusLineage':
#         sum_freq_each_samples.append(0)
        continue
    sum_freq_each_samples.append(sum(freq_df_ordered.loc[:,col].to_list()))
    
density_df = pd.DataFrame(
    columns=freq_df_ordered.columns.to_list()[:-1],
    index=freq_df_ordered.index.to_list())
import numpy as np
for col,sum_col in zip(density_df.columns.to_list(), sum_freq_each_samples):
    density = np.array(freq_df_ordered.loc[:,col].to_list()) / sum_col
    density_df[col] = density

In [161]:
# color map
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

import os
os.makedirs('image',exist_ok=True)

for ind, _ in enumerate(density_df.columns.to_list()):
    
    test = density_df.iloc[:,ind].to_list()
    x = np.linspace(0, len(test)-1, len(test))
    y = [0,1]
    x, y = np.meshgrid(x,y)

    z = np.array([test,test])
    plt.contourf(x,y,z, cmap='jet')
    plt.savefig("image/img" + str(ind) + ".png")
    plt.close()